In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import os
import sys

p = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(p)  # Add the folder_path to the Python path

from DecisionTree.ID3Tree import ID3Tree
from DecisionTree.Ruleset import Ruleset
from DecisionTree.Bootstrap_Aggregating import Bagging

In [16]:
iris = load_iris()
X, y = iris.data.tolist(), iris.target.tolist()
feature_names = iris.feature_names

In [17]:
# Combine features and labels
data = [x + [label] for x, label in zip(X, y)]

# Split data
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)

# Define attribute types (all continuous in iris)
type_map = {attr: 'continuous' for attr in feature_names}

In [18]:
print("\nTesting ID3Tree:")
tree = ID3Tree(feature_names, train_data, default=0, type_map=type_map)  # Default to class 0
tree.train()

# Build rules once after training
rules = tree.build_rules()

correct = 0
for row in test_data:
    # Iterate through all rules to find a matching prediction
    pred_label = None
    for rule in rules:
        pred_label = rule.predict(row)
        if pred_label is not None:
            break  # Stop at the first matching rule

    # Use the default class if no rule matches
    if pred_label is None:
        pred_label = 0  # Default to class 0

    if pred_label == row[-1]:
        correct += 1

print(f"Accuracy: {correct / len(test_data):.2f}")


Testing ID3Tree:
Accuracy: 0.91


In [19]:
# Train and test Ruleset
print("\nTesting Ruleset:")
ruleset = Ruleset(feature_names, train_data, 0, type_map)  # Default to class 0
ruleset.train()
correct = 0
for row in test_data:
    pred, _ = ruleset.predict(row)
    if pred == row[-1]:
        correct += 1
print(f"Accuracy: {correct / len(test_data):.2f}")



Testing Ruleset:
Accuracy: 0.98


In [20]:
print("\nTesting Bagging:")
bagging = Bagging(feature_names, train_data, 0, type_map)  # Default to class 0
bagging.train()
correct = 0
for row in test_data:
    pred, _ = bagging.predict(row)
    if pred == row[-1]:
        correct += 1
print(f"Accuracy: {correct / len(test_data):.2f}")


Testing Bagging:
Training classifier #1
Training classifier #2
Training classifier #3
Training classifier #4
Training classifier #5
Training classifier #6
Training classifier #7
Training classifier #8
Training classifier #9
Training classifier #10
Accuracy: 1.00
